In [127]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/banfakenews/LabeledAuthentic-7K.csv
/kaggle/input/banfakenews/Authentic-48K.csv
/kaggle/input/banfakenews/Fake-1K.csv
/kaggle/input/banfakenews/LabeledFake-1K.csv


In [128]:
import itertools
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
# from nltk.tokenize import word_tokenize

In [129]:
!pip install stopwordsiso

In [130]:
import stopwordsiso as stopwords

In [131]:
df1 = pd.read_csv('/kaggle/input/banfakenews/LabeledAuthentic-7K.csv', engine='python', encoding='utf-8')
df1.tail()

,articleID,domain,date,category,source,relation,headline,content,label
7197,8263,jagonews24.com,2018-09-23 11:12:27,National,দেবপুড় ফাঁড়ী ইনচার্জ ইন্সপেক্টর,Related,তালাকের ৫ বছর পর শ্বশুরবাড়িতে প্রবাসীর মরদেহ,কুমিল্লার বুড়িচং উপজেলার এবদারপুর গ্রামে সাবেক...,1.0
7198,8265,jagonews24.com,2018-09-24 13:36:25,National,বান্দরবানে রাজবিহারের অধ্যক্ষ,Related,বান্দরবানে পালিত হচ্ছে মধু পূর্ণিমা,নানা মাঙ্গলিক আয়োজনের মধ্যে দিয়ে বান্দরবানে পা...,1.0
7199,8266,jagonews24.com,2018-09-23 19:36:27,National,নারায়ণগঞ্জ জেলা পুলিশের অতিরিক্ত পুলিশ সুপার,Related,মাজেদাকে ধরিয়ে দিয়ে ১০ হাজার টাকা পুরস্কার নিল...,নারায়ণগঞ্জের সিদ্ধিরগঞ্জে শীর্ষ মাদক ব্যবসায়ী ...,1.0
7200,8267,independent24.com,2018-09-21 14:26:51,National,"ডাক, টেলিযোগাযোগ ও তথ্যপ্রযুক্তিমন্ত্রী",Related,মন্ত্রিসভায় চূড়ান্ত অনুমোদনের অপেক্ষায় টেলিযোগ...,২০ বছরের পুরনো নীতিমালা দিয়েই চলছে দেশের টেলিয...,1.0
7201,8268,bangla.thereport24.com,2018-09-23 12:29:50,Entertainment,writter,Related,ফুল ঝরে পড়ে,মারুফ কামরুল মুখ ভরে বমি করছে চুটকি। আধ ভাঙা ভ...,1.0


In [132]:
df2= pd.read_csv('/kaggle/input/banfakenews/LabeledFake-1K.csv', engine='python', encoding='utf-8')
df2.tail()

,articleID,domain,date,category,source,relation,headline,content,label,F-type
1294,1303,bengalbeats.com,2019-03-31T21:34:42+00:00,Miscellaneous,Reporter,Unrelated,রাজশাহীতে স্বামীর হাঁচির আওয়াজে ঘুমের মধ্যেই স...,রাজশাহীর সাহেব বাজার মোড়ে রাতে ঘুমের মধ্যে হাস...,0,Satire
1295,1304,bengaliviralnews.com,2018-01-16T15:36:12+00:00,Miscellaneous,Reporter,Unrelated,IAS ইন্টারভিউ এ মেয়েটিকে জিজ্ঞসা করা হল মেয়েদে...,IAS পরীক্ষায় এমন এমন অবাক করে দেওয়া প্রশ্ন এসে...,0,Clickbaits
1296,1305,channeldhaka.news,2018-12-03T18:09:08+00:00,Crime,ফেনী ফায়ার স্টেশনের ইনচার্জ,Unrelated,শাওমি মোবাইল বিস্ফোরণে কলেজছাত্রের মৃত্যু!,শাওমি মোবাইল ফোন বিস্ফোরণ হয়ে স্বপ্নীল মজুমদা...,0,Satire
1297,1306,earki.com,"১৭:৩৬, জানুয়ারি ০৭, ২০১৯",Miscellaneous,Reporter,Related,ফেসবুকে যেভাবে নতুন মন্ত্রীদের অভিনন্দন বার্তা...,আপনাকে নতুন মন্ত্রিসভার সদস্যদের অভিনন্দন জানা...,0,Satire
1298,1307,channeldhaka.news,2019-03-09T23:00:05+00:00,International,মুম্বাই বিমানবন্দর সূত্র,Unrelated,মাঝ আকাশে পর্ন দেখার অভিযোগে পাইলট আটক!,বিমান যখন আকাশে তখন আর কোনও দিকে খেয়াল না রেখ...,0,Satire


In [133]:
#merging df1 & df2
df1.drop(columns=['articleID'], inplace = True)
df2.drop(columns=['articleID','F-type'], inplace = True)

In [134]:
dataset = pd.concat([df1,df2], ignore_index=True)
data = dataset.copy()

In [135]:
data.head()

,domain,date,category,source,relation,headline,content,label
0,bd-pratidin.com,2018-09-20 08:16:43,Sports,আফগান ক্রিকেট বোর্ড (এসিবি) প্রধান,Related,হঠাৎ আফগান ক্রিকেট বোর্ড প্রধানের পদত্যাগ,ক্রিকেট বিশ্বের নতুন চমকের নাম আফগানিস্তান। কয়...,1.0
1,jugantor.com,2018-09-20 20:20:20,Sports,Reporter,Related,টস হেরে বোলিংয়ে বাংলাদেশ,এশিয়া কাপের ষষ্ঠ ম্যাচে বাংলাদেশ দলের বিপক্ষে ...,1.0
2,bd24live.com,2018-09-20 16:39:40,National,Reporter,Related,রাজধানীতে বিশেষ অভিযানে আটক ৪৩,রাজধানীতে মাদক বিরোধী বিশেষ অভিযান পরিচালনা কর...,1.0
3,bd24live.com,2018-09-19 18:27:56,National,জননিরাপত্তা বিভাগের সচিব,Related,উস্কানি রোধে নজরদারি থাকবে সামাজিক যোগাযোগ মাধ...,সনাতন ধর্মাবলম্বীদের সবচেয়ে বড় ধর্মীয় উৎসব দুর...,1.0
4,somoynews.tv,2018-09-20 10:15:28,Finance,পেট্রোল পাম্প মালিক সমিতি সভাপতি,Related,"'যেখানে তেল আসত ৭ দিনে, এখন তা আসবে অতি দ্রুত'",ভারত থেকে পাইপ লাইনের মাধ্যমে সরাসরি দেশে জ্বা...,1.0


In [136]:
stop_words = stopwords.stopwords("bn")
# print(stop_words)

In [137]:
data.drop(columns=['domain','date','source'], inplace = True)

In [138]:
data.head()

,category,relation,headline,content,label
0,Sports,Related,হঠাৎ আফগান ক্রিকেট বোর্ড প্রধানের পদত্যাগ,ক্রিকেট বিশ্বের নতুন চমকের নাম আফগানিস্তান। কয়...,1.0
1,Sports,Related,টস হেরে বোলিংয়ে বাংলাদেশ,এশিয়া কাপের ষষ্ঠ ম্যাচে বাংলাদেশ দলের বিপক্ষে ...,1.0
2,National,Related,রাজধানীতে বিশেষ অভিযানে আটক ৪৩,রাজধানীতে মাদক বিরোধী বিশেষ অভিযান পরিচালনা কর...,1.0
3,National,Related,উস্কানি রোধে নজরদারি থাকবে সামাজিক যোগাযোগ মাধ...,সনাতন ধর্মাবলম্বীদের সবচেয়ে বড় ধর্মীয় উৎসব দুর...,1.0
4,Finance,Related,"'যেখানে তেল আসত ৭ দিনে, এখন তা আসবে অতি দ্রুত'",ভারত থেকে পাইপ লাইনের মাধ্যমে সরাসরি দেশে জ্বা...,1.0


In [139]:
data['relation'] = data['relation'].str.lower()
data['category'] = data['category'].str.lower()

In [140]:
data['relation'].unique()

array(['related', 'unrelated'], dtype=object)

In [141]:
#Preprocess function
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [142]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [143]:
data['headline'] = data['headline'].apply(round1)
data['headline'] = data['headline'].apply(round2)

In [144]:
data['content'] = data['content'].apply(round1)
data['content'] = data['content'].apply(round2)

In [145]:
# Pepocessing function for headline

def cleaning_headline(data):
    
    data['headline'] = data['headline'].str.replace('[A-Za-z]','')
    data['headline'] = data['headline'].str.replace('[।,.॥]','')
    data['headline'] = data['headline'].str.replace('[\xa0]','')
    data['headline'] = data['headline'].str.split(' ')
    data['headline'] = data['headline'].apply(lambda x: [item for item in x if item not in stop_words])
    data['headline'] = data['headline'].apply(lambda x: ' '.join(word for word in x))
  
    return data

data = cleaning_headline(data)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


In [146]:
# Pepocessing function for  content

def cleaning_content(data):
    
    data['content'] = data['content'].str.replace('[A-Za-z]','')
    data['content'] = data['content'].str.replace('[।,.॥]','')
    data['content'] = data['content'].str.replace('[\xa0]','')
    data['content'] = data['content'].str.split(' ')
    data['content'] = data['content'].apply(lambda x: [item for item in x if item not in stop_words])
    data['content'] = data['content'].apply(lambda x: ' '.join(word for word in x))
  
    return data

data = cleaning_content(data)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


In [147]:

import re

def demoji_cont(text):
    
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U00010000-\U0010ffff"
                              "]+", flags=re.UNICODE)
    
    return(emoji_pattern.sub(r'', text))
 
 

data['content'] = data['content'].apply(lambda x:demoji_cont(x))

In [148]:
import re

def demoji_head(text):
    
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U00010000-\U0010ffff"
                              "]+", flags=re.UNICODE)
    
    return(emoji_pattern.sub(r'', text))
 
 

data['headline'] = data['headline'].apply(lambda x:demoji_head(x))

In [149]:
data.head()

,category,relation,headline,content,label
0,sports,related,হঠাৎ আফগান ক্রিকেট বোর্ড প্রধানের পদত্যাগ,ক্রিকেট বিশ্বের চমকের নাম আফগানিস্তান বছরে পার...,1.0
1,sports,related,টস হেরে বোলিংয়ে বাংলাদেশ,এশিয়া কাপের ষষ্ঠ ম্যাচে বাংলাদেশ দলের বিপক্ষে ...,1.0
2,national,related,রাজধানীতে অভিযানে আটক,রাজধানীতে মাদক বিরোধী অভিযান পরিচালনা ঢাকা মেট...,1.0
3,national,related,উস্কানি রোধে নজরদারি সামাজিক যোগাযোগ,সনাতন ধর্মাবলম্বীদের সবচেয়ে বড় ধর্মীয় উৎসব দুর...,1.0
4,finance,related,তেল আসত দিনে আসবে অতি দ্রুত,ভারত পাইপ লাইনের সরাসরি দেশে জ্বালানি তেল সরবর...,1.0


In [150]:
dataset['content'][50]

"নিজেকে কীভাবে শিরোনামে রাখা যায় তা বেশ ভালোই জানেন পুনম পান্ডে। বুধবার ছিল ভারত-পাকিস্তানের ক্রিকেট ম্যাচ। আর সেই ম্যাচের আগেই সোশ্যাল সাইটে নিজের খোলামেলা ছবি শেয়ার করলেন পুনম। যেখানে তাঁর শরীরের কিছু অংশে পর্যাপ্ত সুতোর দেখাও মেলেনি। শরীরের নিন্মাংশ প্রদর্শন করে সোশ্যাল সাইটে উত্তাপ ছড়াতে শুরু করেন পুনম পান্ডে।\xa0 শুধু তাই নয়, এবারই মজা হবে বলেও নিজের ছবির নীচে ক্যাপশন দিতেও দেখা যায় পুনমকে। ‘ইন্টারনেট সেনসেশন’ পুনমের সেই ছবি প্রকাশ্যে আসতেই তা ভাইরাল হয়ে যায় সঙ্গে সঙ্গে। \xa0 তবে এই প্রথমবার নয়, এর আগেও বহুবার নিজের সোশ্যাল হ্যান্ডেলে একাধিক ছবি শেয়ার করে ভক্তদের নজর কেড়েছেন পুনম পান্ডে। কখনও ওই ছবি নিয়ে শুরু হয়েছে জোরদার বিতর্ক, আবার কখনও ভক্তদের হৃদয় জয় করে নিয়েছেন তিনি। আবার কখনও নিজের নামে আপ এনে সমালোচনার কেন্দ্রবিন্দুতে উঠে এসেছেন পুনম। \xa0 যদিও আপটি প্রকাশিত হওয়ার পরই তা তড়িঘড়ি বন্ধ করে দেয়া হয়। যা নিয়ে নিজের মতামতও প্রকাশ করেন পুনম। তা সত্ত্বেও ওই আপকে নতুন করে আর প্রকাশ করতে পারেননি বলিউডের এই নায়িকা। \xa0 প্রসঙ্গত, তবে বর্তমানে 'লেডি গব্বর সিং' নামে একটি তেলুগু সিনেমার 

In [151]:
data['content'][50]

'নিজেকে কীভাবে শিরোনামে যায় ভালোই জানেন পুনম পান্ডে বুধবার ভারতপাকিস্তানের ক্রিকেট ম্যাচ ম্যাচের সোশ্যাল সাইটে খোলামেলা ছবি শেয়ার পুনম শরীরের অংশে পর্যাপ্ত সুতোর দেখাও মেলেনি শরীরের নিন্মাংশ প্রদর্শন সোশ্যাল সাইটে উত্তাপ ছড়াতে পুনম পান্ডে এবারই মজা বলেও ছবির নীচে ক্যাপশন দিতেও যায় পুনমকে ইন্টারনেট সেনসেশন পুনমের ছবি প্রকাশ্যে আসতেই ভাইরাল হয়ে যায়  প্রথমবার আগেও বহুবার সোশ্যাল হ্যান্ডেলে একাধিক ছবি শেয়ার ভক্তদের নজর কেড়েছেন পুনম পান্ডে ছবি হয়েছে জোরদার বিতর্ক ভক্তদের হৃদয় জয় নিয়েছেন নামে আপ এনে সমালোচনার কেন্দ্রবিন্দুতে উঠে এসেছেন পুনম  আপটি প্রকাশিত হওয়ার পরই তড়িঘড়ি বন্ধ দেয়া মতামতও প্রকাশ পুনম সত্ত্বেও আপকে প্রকাশ পারেননি বলিউডের নায়িকা  প্রসঙ্গত বর্তমানে লেডি গব্বর সিং নামে তেলুগু সিনেমার শুটিং পুনম পান্ডে ইত্তেফাকএসজেএ'

In [152]:
X = data.drop(columns=['label','category','relation']).values
y = data.iloc[:,-1].values

In [153]:
X[:,0]

array(['হঠাৎ আফগান ক্রিকেট বোর্ড প্রধানের পদত্যাগ',
       'টস হেরে বোলিংয়ে বাংলাদেশ', 'রাজধানীতে অভিযানে আটক ', ...,
       'শাওমি মোবাইল বিস্ফোরণে কলেজছাত্রের মৃত্যু',
       'ফেসবুকে যেভাবে মন্ত্রীদের অভিনন্দন বার্তা জানাবেন',
       'মাঝ আকাশে পর্ন দেখার অভিযোগে পাইলট আটক'], dtype=object)

In [154]:
#Import the CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000)
News_body = cv.fit_transform(X[:,1]).todense()

In [155]:
News_body.shape

(8501, 5000)

In [156]:
cv_head = CountVectorizer(max_features=5000)
News_head = cv_head.fit_transform(X[:,0]).todense()

In [157]:
News_head.shape

(8501, 2240)

In [158]:
X_mat = np.hstack(( News_head, News_body))

In [159]:
X_mat.shape

(8501, 7240)

In [160]:

type(X_mat)

numpy.matrix

In [161]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

In [162]:
col_transform = make_column_transformer((OneHotEncoder(sparse=False),['category','relation']),
                                        remainder= 'passthrough')

In [163]:
dum = data.drop(columns=['headline','content','label'])
dummi = col_transform.fit_transform(dum)

In [164]:
dummi.shape

(8501, 14)

In [165]:
X_final = np.hstack(( dummi, X_mat))

In [166]:
X_final.shape

(8501, 7254)

In [167]:
X_final

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [168]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final,y, test_size=0.2, random_state=0)

In [169]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(6800, 7254)
(1701, 7254)
(6800,)
(1701,)


In [170]:
from sklearn.ensemble import RandomForestClassifier
nb = RandomForestClassifier()


In [171]:
nb.fit(X_train,y_train)

RandomForestClassifier()

In [172]:
y_pred = nb.predict(X_test)

In [173]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

0.9729570840681951

In [174]:
nb.score(X_train,y_train)

1.0

In [175]:
# Precision and Recall
from sklearn import metrics
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 224,   44],
       [   2, 1431]])

In [176]:
print(metrics.classification_report(y_test,y_pred,digits=3))

              precision    recall  f1-score   support

         0.0      0.991     0.836     0.907       268
         1.0      0.970     0.999     0.984      1433

    accuracy                          0.973      1701
   macro avg      0.981     0.917     0.946      1701
weighted avg      0.973     0.973     0.972      1701



In [177]:
import pickle
pickle.dump(nb, open("random_forest_classi.pkl",'wb'))

# Testing

In [178]:
category = 'national'
relation = 'unrelated'
headline = "মাহির মন ভালো করার বিচিত্র উপায়"
content = '''
অভিনেত্রী মাহিয়া মাহির সেদিন মন খারাপ ছিল। নিজের গাড়িটা নিয়ে বেড়াতে বের হয়ে যান তিনি। গন্তব্য রাজশাহী শহর থেকে গ্রামের দিকে। ইচ্ছা ছিল চোখ ভরে একটু প্রকৃতি দেখবেন। সে রকম পরিকল্পনা নিয়েই ঘুরতে বের হন তিনি। পথেই দেখা হয়ে যায় এক কৃষকের সঙ্গে, যিনি বাঁধাকপি নিয়ে বাজারে বিক্রির উদ্দেশে যাচ্ছিলেন। বয়স্ক মানুষটাকে দেখে তাঁর মন আরও খারাপ হয়। রিকশা ভ্যান থেকে সব কটি বাঁধাকপি কিনে বয়স্ক বিক্রেতার মন ভালো করে দেন মাহি। অন্যের মুখে হাসি ফুটলে তাঁর নিজেরও মন ভালো হয়ে যায়।
মাহির ফেসবুক স্ট্যাটাস দেখে অনেকেই জেনেছেন, ১০০টি বাঁধাকপি কিনেছেন তিনি। কিন্তু এ রকম ঘটনা এবারই প্রথম নয়। আগেও ফেরিওয়ালাদের কাছ থেকে সব পণ্য কিনে নিয়েছেন মাহি। মন খারাপ হলে এমন কাজ আগেও করেছেন তিনি। সেসবের মধ্যে ছিল চুড়ি, গলার মালা, ঝুড়িসহ আরও অনেক কিছু। তিনি জানান, একবার মন খারাপ নিয়ে গুলশানে ঘুরছিলেন তিনি। 
এক সিগন্যালে দেখতে পান, একজন অনেকগুলো ঝুড়ি নিয়ে বসে আছেন। ওই ঝুড়িওয়ালার কাছ থেকে সব কটি ঝুড়ি কিনে নেন তিনি। অতগুলো ঝুড়ি দেখে তাজ্জব হয়ে যান মাহির মা। 
তিনি বলেন, ‘মন খারাপ হলে আমি কারও না কারও সাহায্যে এগিয়ে যাই। আমাকে দিয়ে যদি কারও উপকার হয়, তাতে আমি শান্তি পাই।’ একবার পথশিশুদের ১০০ হাওয়াই মিঠাই কিনে দিয়েছিলেন। সেদিনের শিশুগুলোর আনন্দের ছবি আজও তাঁর চোখে ভাসে। আরেকবার কক্সবাজারের সমুদ্রসৈকতে এক কিশোরী কানের দুল ও গলার মালা বিক্রি করছিল। সারা দিন তেমন কিছু বিক্রি হয়নি বলে মন খারাপ করে ছিল সে। মাহি তার কাছ থেকে সবকিছু কিনে তাকেই উপহার দিয়েছিলেন। সম্প্রতি কেনা ১০০ বাঁধাকপি কী করলেন, জানতে চাইলে মাহি বলেন, ‘কিছু রান্না করেছি, কিছু বাড়ির চারপাশের সবাইকে বিলিয়ে দিয়েছি। যদিও এখানকার মানুষেরা এখন আর এগুলো খায় না। সেসব গরু-ছাগলকে খাইয়েছেন।’
সম্প্রতি মাহি শেষ করেছেন ‘লাইভ’ ছবির শুটিং। এ ছবিতে তাঁর সঙ্গে দেখা যাবে অভিনেতা সাইমনকে। এ ছাড়া সম্প্রতি ‘যাও পাখি বলো তারে’ নামে আরও একটি ছবিতে কাজ করেছেন এই অভিনেত্রী। ছবিটির গানের শুটিং এখনো বাকি রয়েছে। লোকেশন দেখা শেষ হলেই গানগুলোর শুটিংয়ে অংশ নেবেন তিনি। এ ছবিতে কেন্দ্রীয় পুরুষ চরিত্রে অভিনয় করেছেন শিপন মিত্র। মাহির হাতে রয়েছে আরও বেশ কিছু ছবির কাজ। সেগুলোর মধ্যে ‘গ্যাংস্টার’ ছবির শুটিংয়ের জন্য শিগগিরই কক্সবাজার যাবেন তিনি। 

'''

In [179]:
inputs = pd.DataFrame([[category,relation,headline,content]], columns=['category','relation','headline','content'])

In [180]:
inputs

,category,relation,headline,content
0,national,unrelated,মাহির মন ভালো করার বিচিত্র উপায়,\nঅভিনেত্রী মাহিয়া মাহির সেদিন মন খারাপ ছিল। ন...


In [181]:
inputs['relation'] = inputs['relation'].str.lower()
inputs['category'] = inputs['category'].str.lower()

In [182]:
inputs['headline'] = inputs['headline'].apply(round1)
inputs['headline'] = inputs['headline'].apply(round2)
inputs['content'] = inputs['content'].apply(round1)
inputs['content'] = inputs['content'].apply(round2)

In [183]:
inputs = cleaning_headline(inputs)
inputs = cleaning_content(inputs)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


In [184]:
inputs['content'] = inputs['content'].apply(lambda x:demoji_cont(x))
inputs['headline'] = inputs['headline'].apply(lambda x:demoji_head(x))

In [185]:
testing = inputs.drop(columns=['category','relation']).values

In [186]:
testing[:,0]

array(['মাহির মন ভালো বিচিত্র উপায়'], dtype=object)

In [187]:
test_body = cv.transform(testing[:,1]).todense()

In [188]:
test_body.shape

(1, 5000)

In [189]:
test_head = cv_head.transform(testing[:,0]).todense()

In [190]:
test_head.shape

(1, 2240)

In [191]:
test_final = np.hstack(( test_head, test_body))

In [192]:
test_final.shape

(1, 7240)

In [195]:
test_dum = inputs.drop(columns=['headline','content'])
test_dumi = col_transform.transform(test_dum)

In [196]:
test_dumi.shape

(1, 14)

In [197]:
lets_test = np.hstack(( test_dumi, test_final))

In [198]:
lets_test.shape

(1, 7254)

In [ ]:
nb.predict(lets_test)